# Análisis de Inversión Inmobiliaria en CABA

**Objetivo:** Identificar oportunidades de inversión y entender las dinámicas de valor en el mercado inmobiliario de Buenos Aires.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os
import sys
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

warnings.simplefilter(action='ignore', category=FutureWarning)

load_dotenv(dotenv_path='../.env')

def get_data_from_db():
    try:
        db_user = os.getenv("DB_USER")
        db_password = os.getenv("DB_PASSWORD")
        db_host = os.getenv("DB_HOST")
        db_name = os.getenv("DB_NAME")
        db_uri = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}"
        engine = create_engine(db_uri)
        query = "SELECT barrio, price_usd, superficie_total_m2, description FROM propiedades WHERE price_usd IS NOT NULL AND superficie_total_m2 > 0;"
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"Error: {e}")
        return None

df = get_data_from_db()

if df is not None:
    print(f"Se cargaron {len(df)} registros.")


## 1. Análisis de "Joyas Ocultas": Precio/m² vs. Amenities

In [ ]:
if df is not None:
    df['precio_m2'] = df['price_usd'] / df['superficie_total_m2']

    amenities = ['pileta', 'gimnasio', 'sum', 'parrilla', 'seguridad']
    for amenity in amenities:
        df[amenity] = df['description'].str.contains(amenity, case=False, na=False).astype(int)

    df_barrios = df.groupby('barrio').agg(
        precio_m2_promedio=('precio_m2', 'mean'),
        con_amenities_promedio=('pileta', 'mean'), # Usamos pileta como proxy de amenities
        cantidad_propiedades=('barrio', 'size')
    ).reset_index()

    df_filtrado = df_barrios[df_barrios['cantidad_propiedades'] > 100]

    plt.figure(figsize=(14, 10))
    sns.scatterplot(data=df_filtrado, x='precio_m2_promedio', y='con_amenities_promedio', size='cantidad_propiedades', hue='barrio', sizes=(100, 2000), legend=False)

    for i, row in df_filtrado.iterrows():
        plt.text(row['precio_m2_promedio'], row['con_amenities_promedio'], row['barrio'], fontsize=9)

    plt.title('Precio/m² vs. % de Propiedades con Amenities por Barrio', fontsize=16)
    plt.xlabel('Precio Promedio por m² (USD)', fontsize=12)
    plt.ylabel('% de Propiedades con Pileta (como proxy de amenities)', fontsize=12)
    plt.grid(True)
    
    output_dir = "../docs/images"
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(os.path.join(output_dir, "joyas_ocultas.png"))
    plt.show()
